# Predict

In [1]:
from predict import predict_image

predict_image(input_file='/home/eolika/Desktop/BlueMind/nnUNet/brain_ich_1.nii.gz',
              output='/home/eolika/Desktop/BlueMind/nnUNet/brain_ich_predicted.nii.gz',
              model='/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_results/nnUNetTrainerDiceCELoss_noSmooth_500epochs__nnUNetPlans__3d_fullres')

Resampling...
Original spacing: (0.493, 0.493, 1.0)
from shape (512, 512, 186) to shape (168, 168, 124)
Resampled in 2.03s

Loaded resampled image with shape (124, 168, 168) and spacing: [1.5, 1.5, 1.5]

Predicting...
preprocessing
old shape: (124, 168, 168), new_shape: [124 168 168], old_spacing: [1.5, 1.5, 1.5], new_spacing: [1.5, 1.5, 1.5], fn_data: functools.partial(<function resample_data_or_seg_to_shape at 0x7f8eca233400>, is_seg=False, order=3, order_z=0, force_separate_z=None)
predicting
Input shape: torch.Size([1, 124, 168, 168])
step_size: 0.5
mirror_axes: None
n_steps 8, image size is torch.Size([124, 168, 168]), tile_size [112, 128, 128], tile_step_size 0.5
steps:
[[0, 12], [0, 40], [0, 40]]
preallocating arrays
running prediction


100%|██████████| 8/8 [00:00<00:00, 12.04it/s]


Prediction done, transferring to CPU if needed
resampling to original shape
Predicted in 2.09s

Resampling to original spacing...
from shape (168, 168, 124) to shape (512, 512, 186)
Resampled in 0.47s

Done in 6.08s


### batch prediction

In [ ]:
import os
import natsort
from tqdm import tqdm
from predict import predict_image

dataset = "/media/eolika/BlueMindAI/CT/dataset/plethora_dataset"
output_mask_name = "total"
model='/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_results/nnUNetTrainerDiceCELoss_noSmooth_500epochs__nnUNetPlans__3d_fullres'

cases = natsort.natsorted([case for case in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, case))])

pbar = tqdm(cases)

for case in pbar:

    pbar.set_description_str(case)

    ct_file_path = os.path.join(dataset, case, 'ct.nii.gz')
    output_dir = os.path.join(dataset, case, 'segmentations')

    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    predict_image(input_file=ct_file_path,
              output=f'{output_dir}/{output_mask_name}.nii.gz',
              model=model,
              nnunet_verbose=False)

### Get sub-masks from total mask

In [ ]:
import nibabel as nib
import numpy as np
from tqdm import tqdm
import natsort
import os

dataset = "/media/eolika/BlueMindAI/CT/dataset/plethora_dataset"
total_mask_name = 'total'

labels = {
    "bones": 1,
    "arteries": 2,
    "veins": 3,
    "muscles": 4,
    "spleen": 5,
    "kidney_right": 6,
    "kidney_left": 7,
    "gallbladder": 8,
    "liver": 9,
    "liver_vessels": 10,
    "stomach": 11,
    "pancreas": 12,
    "adrenal_gland_right": 13,
    "adrenal_gland_left": 14,
    "esophagus": 15,
    "brain": 16,
    "small_bowel": 17,
    "duodenum": 18,
    "colon": 19,
    "urinary_bladder": 20,
    "lung_upper_lobe_left": 21,
    "lung_lower_lobe_left": 22,
    "lung_upper_lobe_right": 23,
    "lung_middle_lobe_right": 24,
    "lung_lower_lobe_right": 25,
    "lung_trachea_bronchia": 26,
    "lung_vessels": 27,
    "heart_myocardium": 28,
    "heart_atrium_left": 29,
    "heart_ventricle_left": 30,
    "heart_atrium_right": 31,
    "heart_ventricle_right": 32,
    "liver_formation": 33,
    "lung_formation": 34,
    "kidney_formation": 35,
    "pleural_effusion": 36,
    "intracerebral_hemorrhage": 37
}

cases = natsort.natsorted([case for case in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, case))])

pbar = tqdm(cases)

for case in pbar:

    pbar.set_description_str(case)

    ct_file_path = os.path.join(dataset, case, 'ct.nii.gz') 
    output_dir = os.path.join(dataset, case, 'segmentations')
    mask_file_path = os.path.join(output_dir, 'total.nii.gz') 

    ct_img = nib.load(ct_file_path)
    mask_img = nib.load(mask_file_path)
    mask_data = mask_img.get_fdata()

    for label, indx in labels.items():
        label_mask = np.array(mask_data == indx).astype(np.uint8)
        nib.save(nib.Nifti1Image(label_mask, mask_img.affine), os.path.join(output_dir, label+'.nii.gz'))